In [1]:
import model.ChainCRF
import importlib
import dataLoad.lload

importlib.reload(model.ChainCRF)
importlib.reload(dataLoad.lload)
from dataLoad.lload import CustomDataset
from dataLoad.lload import *
import numpy as np
from model.ChainCRF import BLITM, ChainCRF
import torchtext
from get_data import *
from check import *
from collections import Counter, defaultdict

language = "Chinese"
mode = False
hidden_dim = 100 if language == "English" else 100
embed_size = 100 if language == "English" else 200
for_test = True
min_freq = 4 if language == "English" else 4
##Chinese 300 100 10,1e-3  Loss:281.3659775416056 micro avg  0.9368 
##Chinese 100 100 10,1e-3  Loss:281.3659775416056 micro avg  0.9447 
##Chinese 50 100 10,1e-4  Loss:281.3659775416056 micro avg 0.6839 
##Chinese 50 100 10,5e-4  Loss:277.4861424763997 micro avg 0.9065
## 0.7683 
## English
##  pretrain 8317 +0.8276 =0.8320  1e-3 10
##  pretrain 8317 +0.8276 =0.8347  1e-3 10 zeros
##  pretrain 8317 +0.8276 =0.8399  1e-4 10 zeros
#   0.8381 


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class BiLSTM_CRF(nn.Module):
    def __init__(self, num_classes, vocab_length, embedding_dim, hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        self.bilstm = BLITM(num_classes, vocab_length, embedding_dim, hidden_dim)
        self.crf = ChainCRF(num_classes)

    def forward(self, sentence, mask, targets=None, pre_train=None):

        emissions = self.bilstm(sentence)
        if targets is not None:
            # 计算CRF损失
            # mask = (sentence != 0)  # 使用 0 填充的词的位置作为掩码
            crf_loss = self.crf(emissions, targets, mask)
            return crf_loss
        else:
            # 测试时，使用维特比解码
            tags = self.crf.viterbi_decode(emissions, mask)
            return tags


In [3]:

train_data = get_train_data(language)
valid_data = get_valid_data(language)

import os
import torchtext

train_word = [word for sentence in train_data for word, label in sentence]
print(len(train_word))

vocab1 = torchtext.vocab.vocab(Counter(train_word), min_freq=min_freq, specials=['<unk>'])
vocab1.set_default_index(vocab1['<unk>'])
train_data = train_data + valid_data
train_word = [word for sentence in train_data for word, label in sentence]
print(len(train_word))

vocab2 = torchtext.vocab.vocab(Counter(train_word), min_freq=min_freq, specials=['<unk>'])
vocab2.set_default_index(vocab2['<unk>'])
print(vocab2.get_default_index())
if for_test:
    vocab = vocab2
else:
    vocab = vocab1


# vocab不变
def sent2word(sentence):
    return [w for w, _ in sentence]


def sent2label(sentence):
    return [l for _, l in sentence]


max_length = max([len(l) for l in train_data])
max_length = max(max_length, 256)
sorted_labels = sorted_labels_chn if language == 'Chinese' else sorted_labels_eng


def label2index(label):
    return sorted_labels.index(label)


print(len(vocab))

123967
137849
0
1148


In [4]:

# Create a custom dataset
custom_dataset = CustomDataset(train_data, vocab, label2index, max_length)
# Create a DataLoader
batch_size = 32
dataloader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True)

custom_dataset[0]

{'word_embeddings': tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
         19,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         

In [5]:
import torch
import torch.optim as optim
from tqdm import tqdm


def train(model, train_loader, num_epochs, learning_rate, device):
    # Move the model to GPU
    model.to(device)

    # Define the optimizer
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Create a progress bar
    progress_bar = tqdm(total=num_epochs * len(train_loader))

    for epoch in range(num_epochs):
        model.train()
        sum_loss = 0

        for batch in train_loader:
            # Move inputs, labels, and mask to GPU
            length = batch['max_length']
            max_length = np.argmax(length)
            aaa = length[max_length]
            inputs = batch['word_embeddings'][:, :aaa].to(device)
            labels = batch['label_indices'][:, :aaa].to(device)
            mask = batch['mask'].to(device)[:, :aaa].to(device)

            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass
            loss = model(inputs, mask, labels)

            # Backward pass
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.1)
            sum_loss += loss.item()

            # Update the parameters
            optimizer.step()
            progress_bar.update(1)

        progress_bar.set_postfix_str("Epoch:{}, Loss:{}".format(epoch + 1, sum_loss / len(train_loader)))
        print(" ")

    # Close the progress bar
    progress_bar.close()

# Example usage:
# Assuming you have a model, train_loader, num_epochs, and learning_rate



In [6]:
bilstm_crf = BiLSTM_CRF(len(sorted_labels), len(vocab), embed_size, hidden_dim)
pretrain_file = "./bilstm_crf/pretrain/BILSTM_{}.bin".format(language)
if for_test:
    save_file = "./weight/bilstm/BILSTM_CRF_{}_final__temp.bin".format(language)
else:
    save_file = "./weight/bilstm/BILSTM_CRF_{}__temp.bin".format(language)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
load_file1 = "./weight/bilstm/BILSTM_CRF_{}.bin".format(language)

load_file2 = "./weight/bilstm/BILSTM_CRF_{}_final.bin".format(language)
if mode:
    train(bilstm_crf, dataloader, 10, 1e-3, device)
    torch.save(bilstm_crf.state_dict(), save_file)
else:
    if for_test:
        load_file = load_file2
    else:
        load_file = load_file1
    bilstm_crf.load_state_dict(torch.load(load_file))

In [7]:
def mycheck(language, vocab, res_file, model, train_or_valid, device):
    valid = get_data_from_file(res_file)
    pred_path = "example_data/BILSTM_CRF_{}_{}.txt".format(
        language, train_or_valid)
    valid_data = CustomDataset(valid, vocab, label2index, 256)
    valdataloader = DataLoader(valid_data, batch_size=64, shuffle=False)

    # Move the model to GPU
    # device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    with open(pred_path, "w", encoding='utf-8') as f:
        with torch.no_grad():
            iter = 0
            for batch in valdataloader:
                # Move inputs, labels, and mask to GPU
                length = batch['max_length']
                max_length = np.argmax(length)
                aaa = length[max_length]
                word_embeddings = batch['word_embeddings'][:, :aaa]
                masks = batch['mask'][:, :aaa]

                preds = model(word_embeddings, masks)

                for pred in preds:
                    pred_labels = []
                    for i in range(len(valid[iter])):
                        f.write(valid[iter][i][0] + " " +
                                sorted_labels[pred[i]] + '\n')
                        pred_labels.append(sorted_labels[pred[i]])
                    f.write('\n')
                    iter = iter + 1

    # Move the model back to CPU if needed
    model.to("cpu")

    res = check(language, res_file, pred_path)
    return res


def test(language, res_file, device):
    bilstm_crf2 = BiLSTM_CRF(len(sorted_labels), len(
        vocab2), embed_size, hidden_dim)
    if mode:
        bilstm_crf2.load_state_dict(torch.load(save_file))
    else:

        bilstm_crf2.load_state_dict(torch.load(load_file2))
    # res1 = mycheck(language=language,vocab=vocab1,res_file=res_file,model=bilstm_crf1,train_or_valid="test",device=device)
    res2 = mycheck(language=language, vocab=vocab2, res_file=res_file,
                   model=bilstm_crf2, train_or_valid="test", device=device)


In [8]:
device = torch.device("cpu")

#mycheck(language=language,vocab=vocab,res_file="{}/train.txt".format(language),model=bilstm_crf,train_or_valid="train",device=device)
#mycheck(language=language,vocab=vocab,res_file="{}/validation.txt".format(language),model=bilstm_crf,train_or_valid="valid",device=device)
#language train val
#chinese 0.9912  0.9497
#english 0.9898  0.8205

In [9]:

test(language=language, res_file="test/{}/test.txt".format(language), device=device)
# 0.7519 
# 0.9461


Detected encoding for gold_path: utf-8
Detected encoding for my_path: utf-8
              precision    recall  f1-score   support

      B-NAME     1.0000    0.9107    0.9533       112
      M-NAME     0.9750    0.9512    0.9630        82
      E-NAME     0.9906    0.9375    0.9633       112
      S-NAME     0.0000    0.0000    0.0000         0
      B-CONT     1.0000    1.0000    1.0000        28
      M-CONT     1.0000    1.0000    1.0000        53
      E-CONT     1.0000    1.0000    1.0000        28
      S-CONT     0.0000    0.0000    0.0000         0
       B-EDU     0.9735    0.9821    0.9778       112
       M-EDU     0.9884    0.9553    0.9716       179
       E-EDU     0.9910    0.9821    0.9865       112
       S-EDU     0.0000    0.0000    0.0000         0
     B-TITLE     0.9385    0.9312    0.9348       770
     M-TITLE     0.9358    0.9099    0.9227      1921
     E-TITLE     0.9819    0.9844    0.9831       770
     S-TITLE     0.0000    0.0000    0.0000         0
     